In [1]:
pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Things to import
import pandas as pd 
import wikipedia as wp
import requests
import io
from bs4 import BeautifulSoup

# Scrape this wikipedia page
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")

# Table index
df_postcode = pd.read_html(html, header = 0)[0]

# Ignore the cells with a borough that is Not assigned.
df_postcode = df_postcode[df_postcode.Borough != 'Not assigned']

df_postcode = df_postcode.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

for index, row in df_postcode.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

In [3]:
# Data obtaining imports
import requests
import io

# Obtain longitude and lattitudes
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

# Merge geospatial data column with the dataframe
c.columns = ['Postcode', 'Latitude', 'Longitude']
df_postcode = pd.merge(c, df_postcode, on='Postcode')

df_postcode = df_postcode[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]

In [4]:
# map rendering library
import folium 
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [6]:
latitude = 43.70011
longitude = -79.41630
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.70011, -79.4163.


In [8]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_postcode['Latitude'], df_postcode['Longitude'], df_postcode['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto